# Lab 6

**Firstname Lastname**

**Submission instructions**

Submit the Colab link to your notebook in Canvas. In addition, take screenshots of the map for each question and submit them to Canvas as well.

In [1]:
import ee
import geemap
geemap.ee_initialize()
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Question 1

Create a map to visualize [NOAA GFS Temperature Data](https://developers.google.com/earth-engine/datasets/catalog/NOAA_GFS0P25) and add a color bar and [NOAA logo](https://i.imgur.com/spILFEi.png) to the map. 

In [10]:
# Add NOAA GFS
dataset = ee.ImageCollection('NOAA/GFS0P25') \
                  .filter(ee.Filter.date('2018-03-01', '2018-03-02'))
temperatureAboveGround = dataset.select('temperature_2m_above_ground')
visParams = {
  'min': -40.0,
  'max': 35.0,
  'palette': ['blue', 'purple', 'cyan', 'green', 'yellow', 'red'],
}
Map.setCenter(71.72, 52.48, 3.0)
Map.addLayer(temperatureAboveGround, visParams, 'Temperature Above Ground')



In [11]:
# Add a logo to the map
logo = 'https://i.imgur.com/spILFEi.png'
Map.add_image(logo, position='bottomleft')

# Add colorbar to the map
Map.add_colorbar(visParams, label="Above Ground TEmperature March 2018", layer_name="temeratureAboveGround")
Map

Map(bottom=972.0, center=[52.48, 71.72], controls=(WidgetControl(options=['position', 'transparent_bg'], widge…

![](https://i.imgur.com/bBoRswR.png)

## Question 2

**Linked Maps:** Create a 2*2 linked map to visualize the Landsat imagery (`ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')`) with different band combinations.

In [32]:
Map = geemap.Map()
# Add landsat
dataset = ee.ImageCollection('LANDSAT/LE07/C02/T2_TOA') \
    .filterDate('1999-01-01', '2002-12-31')
TrueColor321 = dataset.select(['B3', 'B2', 'B1'])
TrueColor321Vis = {
  'min': 0.0,
  'max': 0.4,
  'gamma': 1.2,
}
Map.setCenter(22.19, 37.67, 6)
Map.addLayer(TrueColor321, TrueColor321Vis, 'True Color (321)')
Map

Map(center=[37.67, 22.19], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

In [ ]:
-43.41,60.83

![](https://i.imgur.com/Xp15uOo.png)

In [33]:
vis_params = [
    {'bands': ['B3', 'B2', 'B1'], 'min': 0, 'max': 0.4, 'gamma': 1.3},
    {'bands': ['B4', 'B5', 'B3'], 'min': 0, 'max': 0.4, 'gamma': 1.3},
    {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.4, 'gamma': 1.3},
    {'bands': ['B5', 'B4', 'B3'], 'min': 0, 'max': 0.4, 'gamma': 1.3},
]

labels = [
    'True Color (321)',
    'Land/Water (453)',
    'Color Infrared (432)',
    'Vegetation (543)',
]

geemap.linked_maps(
    rows=2,
    cols=2,
    height="300px",
    center=[37.67, 22.19],
    zoom=10,
    ee_objects=[dataset],
    vis_params=vis_params,
    labels=labels,
    label_position="topright",
)

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

## Question 3

**Timeseries Inspector:** Create a map with timeseries inspector to visualize [USDA NASS Cropland Data Layers](https://developers.google.com/earth-engine/datasets/catalog/USDA_NASS_CDL) from 2010 to 2022. Add your name and [USDA logo](https://i.imgur.com/tzH2dNr.png) to the map.

In [ ]:
# Add your code here.

![](https://i.imgur.com/gW1XPhL.png)

In [38]:
dataset = ee.ImageCollection('USDA/NASS/CDL') \
                  .filter(ee.Filter.date('2010-01-01', '2022-12-31')) \
                  .first()
cropLandcover = dataset.select('cropland')
Map.setCenter(-100.55, 40.71, 4)
Map.addLayer(cropLandcover, {}, 'Crop Landcover')
logo = 'https://i.imgur.com/tzH2dNr.png'

left_layer = geemap.ee_tile_layer(cropLandcover, {}, 'USDA NASS Cropland Data')
right_layer = geemap.ee_tile_layer(cropLandcover, {}, 'USDA NASS Cropland Data')
Map.split_map(left_layer, right_layer, add_close_button=True)
Map.add_image(logo, position='bottomleft')

Map(bottom=1840.0, center=[40.71, -100.55], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_i…

In [44]:
collection = ee.ImageCollection('USDA/NASS/CDL').select('cropland')
vis_params = {'bands': ['cropland']}
years = collection.aggregate_array('system:index').getInfo()
years

['1997',
 '1998',
 '1999',
 '2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005a',
 '2005b',
 '2006',
 '2007a',
 '2007b',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021',
 '2022']

In [50]:
Map.ts_inspector(
    left_ts=collection,
    right_ts=collection,
    left_names=years,
    right_names=years,
    left_vis=vis_params,
    right_vis=vis_params,
    width='80px',
)
Map

## Question 4

**Time slider:** Create a map with the time slider to visualize [Sentinel-2](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR) for Knoxville, TN.

In [4]:
# Set center to Knoxville, TN, add sentinel, add time slider
 # Function to mask clouds using the Sentinel-2 QA band
def maskS2clouds(image):
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
      .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

  return image.updateMask(mask).divide(10000)

dataset = ee.ImageCollection('COPERNICUS/S2_SR') \
                  .filterDate('2020-01-01', '2020-01-30') \
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)) \
                  .map(maskS2clouds)

visualization = {
  'min': 0.0,
  'max': 0.3,
  'bands': ['B8', 'B4', 'B3'],
}

Map.setCenter(-83.9271, 35.9628, 12)

# Map.addLayer(dataset.mean(), visualization, 'Knoxville, TN, Color Infrared (843)')
# Adding CIR layer works but adding time slider leads to error too many bands. Not sure why
Map.add_time_slider(dataset, visualization, time_interval=2)
Map

EEException: ImageCollection.toBands: Too many bands (more than 5000).

![](https://i.imgur.com/rJe2rvP.png)

## Question 5

**Split-panel Map:** Use the following datasets to create a split-panel map for visualizing the ESA land cover data in the US. Add the ESA land cover legend to the map (Hint: the built-in legend for ESA land cover is `ESA_WorldCover`).

- [US Census States](https://developers.google.com/earth-engine/datasets/catalog/TIGER_2018_States): `ee.FeatureCollection("TIGER/2018/States")`
- [ESA WorldCover 10m](https://developers.google.com/earth-engine/datasets/catalog/ESA_WorldCover_v100): `ee.ImageCollection("ESA/WorldCover/v100")`
- Landsat: `LANDSAT/LE7_TOA_5YEAR/1999_2003`


Currently, the split-map control of ipyleaflet plotting backend has a bug ([source](https://github.com/jupyter-widgets/ipyleaflet/issues/1066)). Use the folium plotting backend instead.

In [10]:
#Did the folium version in leafmap (which has folium)

#esa
esa = ee.ImageCollection('ESA/WorldCover/v100').first()
esaVis = {'bands': ['Map']}

#landsat
landsat = ee.ImageCollection('LANDSAT/LE07/C02/T2_TOA') \
    .filterDate('1999-01-01', '2002-12-31')
TrueColor321 = dataset.select(['B3', 'B2', 'B1'])
TrueColor321Vis = {
  'min': 0.0,
  'max': 0.4,
  'gamma': 1.2,
}



Map.setCenter(-100.55, 40.71, 4)
left_layer = geemap.ee_tile_layer(landsat, TrueColor321Vis, 'landsat')
right_layer = geemap.ee_tile_layer(esa, esaVis, 'ESA Land Cover')
Map.split_map(left_layer, right_layer, add_close_button=True)
Map.add_legend(builtin_legend='ESA_WorldCover')
Map


Map(bottom=681.0, center=[40.71, -100.55], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in…

![](https://i.imgur.com/y0GW6k8.png)

In [13]:
import leafmap.leafmap as leafmap

ModuleNotFoundError: No module named 'leafmap'